## Mermaid Diagrams in a Jupyter Notebook

- This system is running locally on node.js.
- Takes filepaths as input.
- Accepts mermaid diagrams as either regular .mmd or .md (with backticks) 
- Renders diagrams either with or without ELK layout (config in Notebook, not .md file).
- Renders diagrams either TD or LR (config in Notebook, not .md file).
- Renders diagrams as SVG
- Exports from Notebook as PDF, HTML or SVG.

This code is based on <https://mermaid.js.org/config/Tutorials.html#jupyter-integration-with-mermaid-js>.

In [ ]:
import base64
import re
from pathlib import Path

from IPython.display import Image, display


def mm_ink(graphbytes):
    """Return a mermaid.ink URL for the provided bytes."""
    return "https://mermaid.ink/img/" + base64.b64encode(graphbytes).decode(
        "ascii"
    )


def mm_display(graphbytes):
    """Display the mermaid image for the provided bytes."""
    display(Image(url=mm_ink(graphbytes)))


def _strip_wrappers(text):
    """Strip common wrappers so a file can contain a Mermaid diagram in several forms.

    This function supports files containing:
    - a wrapper call around the diagram (for example, mm(...))
    - a fenced code block (triple-backtick block containing the diagram)
    - plain Mermaid source
    """
    if not text:
        return ""

    # Remove leading/trailing wrapper like mm("""...""") or mm('''...''')
    text = re.sub(r"^\s*mm\(\s*(?:\"\"\"|\'\'\')", "", text, flags=re.I)
    text = re.sub(
        r"(?:\"\"\"|\'\'\')\s*\)\s*;?\s*$", "", text, flags=re.I | re.S
    )

    # If file uses a fenced code block, extract inner content
    lines = text.splitlines()
    if lines and lines[0].strip().startswith("```"):
        # find the last fence line
        end_idx = None
        for i in range(len(lines) - 1, -1, -1):
            if lines[i].strip().startswith("```") and i != 0:
                end_idx = i
                break
        if end_idx is not None:
            return "\n".join(lines[1:end_idx]).strip()
        # fallback: return everything except the first line
        return "\n".join(lines[1:]).strip()

    return text.strip()


def mm(graph):
    """Display a mermaid graph (string)."""
    graphbytes = graph.encode("ascii", errors="replace")
    mm_display(graphbytes)


def mm_link(graph):
    """Return a mermaid.ink link for the graph string."""
    graphbytes = graph.encode("ascii", errors="replace")
    return mm_ink(graphbytes)


def mm_path(path):
    """Read a file (path can be str or Path), strip wrappers and display the mermaid diagram."""
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"{p} not found")
    raw = p.read_text(encoding="utf-8")
    mermaid = _strip_wrappers(raw)
    if not mermaid:
        raise ValueError(f"No mermaid content found in {p}")
    graphbytes = mermaid.encode("ascii", errors="replace")
    mm_display(graphbytes)


# Example usage (run this cell first to define helpers):
# mm_path(r"C:\Users\easts\Desktop\digital-mgt\flowchartLR-for-juptyer.mmd")
# mm_path(r"C:\Users\easts\Desktop\digital-mgt\flowchart LR.md")

In [ ]:
import subprocess
import tempfile
import re
from pathlib import Path
from IPython.display import SVG, display

# Try common full paths for mmdc installed by npm on Windows
MMC_PATHS = [
    r"C:\Users\easts\AppData\Roaming\npm\mmdc.cmd",
    r"C:\Users\easts\AppData\Roaming\npm\mmdc.ps1",
    r"C:\Program Files\nodejs\mmdc.cmd",
    r"C:\Program Files\nodejs\mmdc",
]


def find_mmdc_executable():
    """Return an executable path for mmdc if found, else None. Tries common locations first, then 'mmdc' on PATH."""
    import os

    for p in MMC_PATHS:
        if os.path.exists(p):
            return p
    # fallback: try system path
    try:
        from shutil import which

        wp = which("mmdc")
        if wp:
            return wp
    except Exception:
        pass
    return None


def mm_svg_local(graph_or_path, is_path=None, elk=False, orient=None):
    """
    Create and display an SVG version of a Mermaid diagram using the local mmdc CLI.
    This provides optional support for ELK layout and orientation forcing.

    Parameters:
    - graph_or_path: str or Path. If a path (existing file) and is_path is True or auto-detected,
      the file is read. Otherwise graph_or_path is treated as raw Mermaid source.
    - is_path: None|bool. If None, the function will auto-detect whether graph_or_path
      refers to an existing file. If explicitly True/False, that choice is honored.
    - elk: bool|str. If True (or 'true'), the ELK init block will be injected to force ELK
      layout. If False or omitted or an unrecognized value, ELK will not be forced (default False).
    - orient: str|None. If 'lr' or 'td', forces a flowchart orientation.
    """
    # Auto-detect whether graph_or_path is a filesystem path when is_path is None.
    if is_path is None:
        try:
            p = Path(graph_or_path)
            is_path = p.exists()
        except Exception:
            is_path = False

    # Normalize elk to a boolean
    if isinstance(elk, str):
        elk = elk.strip().lower() in ("true", "1", "yes", "y")
    else:
        elk = bool(elk)

    # Normalize orient
    if isinstance(orient, str):
        orient = orient.strip().lower()
        if orient not in ("lr", "td"):
            orient = None
    else:
        orient = None

    # Read source
    if is_path:
        p = Path(graph_or_path)
        if not p.exists():
            raise FileNotFoundError(f"{p} not found")
        raw = p.read_text(encoding="utf-8")
        mermaid_body = _strip_wrappers(raw)
    else:
        mermaid_body = _strip_wrappers(graph_or_path)

    # Inject ELK init block if requested
    if elk:
        has_init = re.search(r"%%\s*{\s*init\s*:", mermaid_body, re.IGNORECASE)
        elk_init = '%%{init: {"flowchart": {"defaultRenderer": "elk", "layout": "elk"}}}%%\n'
        if not has_init:
            mermaid_body = elk_init + mermaid_body
        else:
            mermaid_body = re.sub(
                r"%%\s*{\s*init\s*:[^%]*%%",
                elk_init.strip(),
                mermaid_body,
                flags=re.IGNORECASE,
            )

    # Inject or replace orientation if requested
    if orient:
        # Try to replace an existing flowchart directive (with or without TD/LR)
        # Match 'flowchart' optionally followed by whitespace and TD/LR
        new_body, nsubs = re.subn(
            r"^\s*flowchart(?:\s+(?:lr|td))?",
            f"flowchart {orient.upper()}",
            mermaid_body,
            count=1,
            flags=re.IGNORECASE | re.MULTILINE,
        )
        if nsubs > 0:
            mermaid_body = new_body
        else:
            # If no directive was found to replace, insert one.
            # Place it after any init block for correctness.
            init_match = re.match(r"(%%\s*{\s*init\s*:[^%]*%%.*?\n)", mermaid_body, re.DOTALL | re.IGNORECASE)
            if init_match:
                 # Insert after the init block
                insert_point = init_match.end(1)
                mermaid_body = mermaid_body[:insert_point] + f"flowchart {orient.upper()}\n" + mermaid_body[insert_point:]
            else:
                # Insert at the beginning
                mermaid_body = f"flowchart {orient.upper()}\n" + mermaid_body


    # Write to temp files and call mmdc
    import os

    with tempfile.NamedTemporaryFile(
        mode="w", delete=False, suffix=".mmd", encoding="utf-8"
    ) as infile, tempfile.NamedTemporaryFile(
        mode="r", delete=False, suffix=".svg", encoding="utf-8"
    ) as outfile:
        infile.write(mermaid_body)
        infile_path = infile.name
        outfile_path = outfile.name

    mmdc_exe = find_mmdc_executable()
    if not mmdc_exe:
        print(
            "ERROR: 'mmdc' command not found. Please ensure @mermaid-js/mermaid-cli is installed globally."
        )
        print("You can try running: npm install -g @mermaid-js/mermaid-cli")
        return

    try:
        # Run the Mermaid CLI (mmdc)
        cmd = [
            mmdc_exe,
            "-i", infile_path,
            "-o", outfile_path,
            "-w", "1024",
            "-H", "1024",
        ]
        if mmdc_exe.lower().endswith(".ps1"):
            cmd = ["powershell", "-ExecutionPolicy", "Bypass", "-File"] + cmd
            
        subprocess.run(cmd, check=True, capture_output=True, text=True)

        # Read the generated SVG content
        with open(outfile_path, "r", encoding="utf-8") as f:
            svg_content = f.read()

        # Display the SVG
        display(SVG(svg_content))

    except subprocess.CalledProcessError as e:
        print(
            f"ERROR: mmdc failed to render the diagram. Return code: {e.returncode}"
        )
        print(f"Stderr: {e.stderr}")
        print(f"Stdout: {e.stdout}")
    finally:
        # Clean up temp files
        try:
            os.remove(infile_path)
        except Exception:
            pass
        try:
            os.remove(outfile_path)
        except Exception:
            pass

In [ ]:
print("---")

mm_svg_local(r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\alert_state_machine.md", elk=True, orient="td")


print("---")

# mm_svg_local(r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\data_schema.md", elk=True, orient="td")

print("---")

mm_svg_local(
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\forecast_algo.md",
    elk=True, orient="td"
)


print("---")

mm_svg_local(
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\main_detailed.md",
    elk=True, orient="td"
)


print("---")

mm_svg_local(
    r"C:\Users\easts\github\bmw-sales-forecast\project-documents\py\mermaid_scripts\reporting_flow.md",
    elk=True, orient="td"
)



